In [1342]:
import pandas as pd
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from datetime import date
import seaborn as sns
import matplotlib.pyplot as plt

In [1343]:
# read all tables
customers = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/customers_final.csv')
engagement = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/engagements_final.csv')
marketing = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/marketing_final.csv')
transactions = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/transactions_final.csv')

In [1344]:
# example 1: join at customer level with # of marketing campaigns customer responded YES to
# step 1: marketing data at a customer level

marketing_agg = marketing[marketing['response']=='Yes'].groupby('customer_id')['campaign_id'].count().to_frame()

In [1345]:
# step 2: aggregate transaction data at a customer level
transactions_agg = transactions.groupby('customer_id').aggregate({'transaction_id':'count','transaction_amount':'sum'})

In [1346]:
# step 3: set customers and engagement index as customer_id
customers.set_index('customer_id', inplace=True)
engagement.set_index('customer_id', inplace=True)

In [1347]:
# step 4: join all tables
merged_df = customers.join(engagement).join(transactions_agg).join(marketing_agg)
merged_df

,join_date,last_purchase_date,age,gender,location,number_of_site_visits,number_of_emails_opened,number_of_clicks,transaction_id,transaction_amount,campaign_id
customer_id,,,,,,,,,,,
1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,10,15,1,6,3509.48,1.0
2,2021-09-08,2023-10-25,NaN,Male,Hillville,285,49,51,9,6081.32,2.0
3,2021-06-01,2022-11-27,NaN,NaN,North Latoyatown,192,73,25,6,1454.87,1.0
4,2022-01-01,2022-09-01,29.0,Male,Grossstad,110,30,17,20,7874.68,1.0
5,2022-01-24,2023-06-02,NaN,Male,East Matthewfort,161,2,7,24,15524.55,NaN
...,...,...,...,...,...,...,...,...,...,...,...
9996,2022-12-16,2023-08-13,42.0,Female,Johnstonborough,119,47,16,10,5498.20,NaN
9997,2022-07-09,2023-01-25,26.0,Male,Jessicamouth,3,33,14,12,5848.30,1.0
9998,2023-09-17,2024-01-30,39.0,Male,New John,53,17,5,3,3503.13,1.0


In [1348]:
#Check last transaction date for each customer

last_transaction = transactions.groupby('customer_id')['transaction_date'].max()
last_transaction

customer_id
1        2024-03-12
2        2023-02-26
3        2022-11-18
4        2022-08-31
5        2023-06-01
            ...    
9996     2023-06-11
9997     2022-12-08
9998     2024-01-01
9999     2022-07-14
10000    2023-08-31
Name: transaction_date, Length: 10000, dtype: object

In [1349]:
first_transaction = transactions.groupby('customer_id')['transaction_date'].min()
first_transaction

customer_id
1        2024-01-09
2        2021-09-29
3        2021-09-28
4        2022-01-11
5        2022-03-02
            ...    
9996     2023-01-01
9997     2022-07-15
9998     2023-11-13
9999     2022-05-24
10000    2023-05-27
Name: transaction_date, Length: 10000, dtype: object

In [1350]:
#Add 
merged_df['average_spend'] = merged_df.groupby('customer_id')['transaction_amount'].mean()
merged_df


,join_date,last_purchase_date,age,gender,location,number_of_site_visits,number_of_emails_opened,number_of_clicks,transaction_id,transaction_amount,campaign_id,average_spend
customer_id,,,,,,,,,,,,
1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,10,15,1,6,3509.48,1.0,3509.48
2,2021-09-08,2023-10-25,NaN,Male,Hillville,285,49,51,9,6081.32,2.0,6081.32
3,2021-06-01,2022-11-27,NaN,NaN,North Latoyatown,192,73,25,6,1454.87,1.0,1454.87
4,2022-01-01,2022-09-01,29.0,Male,Grossstad,110,30,17,20,7874.68,1.0,7874.68
5,2022-01-24,2023-06-02,NaN,Male,East Matthewfort,161,2,7,24,15524.55,NaN,15524.55
...,...,...,...,...,...,...,...,...,...,...,...,...
9996,2022-12-16,2023-08-13,42.0,Female,Johnstonborough,119,47,16,10,5498.20,NaN,5498.20
9997,2022-07-09,2023-01-25,26.0,Male,Jessicamouth,3,33,14,12,5848.30,1.0,5848.30
9998,2023-09-17,2024-01-30,39.0,Male,New John,53,17,5,3,3503.13,1.0,3503.13


In [1351]:
#Total metrics of the customers over their entire customer lifetime until the latest date (May 31, 2024)
latest_date = pd.to_datetime('2024-05-31')
merged_df['Customer_age'] = latest_date - pd.to_datetime(merged_df['join_date'])
merged_df


,join_date,last_purchase_date,age,gender,location,number_of_site_visits,number_of_emails_opened,number_of_clicks,transaction_id,transaction_amount,campaign_id,average_spend,Customer_age
customer_id,,,,,,,,,,,,,
1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,10,15,1,6,3509.48,1.0,3509.48,193 days
2,2021-09-08,2023-10-25,NaN,Male,Hillville,285,49,51,9,6081.32,2.0,6081.32,996 days
3,2021-06-01,2022-11-27,NaN,NaN,North Latoyatown,192,73,25,6,1454.87,1.0,1454.87,1095 days
4,2022-01-01,2022-09-01,29.0,Male,Grossstad,110,30,17,20,7874.68,1.0,7874.68,881 days
5,2022-01-24,2023-06-02,NaN,Male,East Matthewfort,161,2,7,24,15524.55,NaN,15524.55,858 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,2022-12-16,2023-08-13,42.0,Female,Johnstonborough,119,47,16,10,5498.20,NaN,5498.20,532 days
9997,2022-07-09,2023-01-25,26.0,Male,Jessicamouth,3,33,14,12,5848.30,1.0,5848.30,692 days
9998,2023-09-17,2024-01-30,39.0,Male,New John,53,17,5,3,3503.13,1.0,3503.13,257 days


In [1352]:
# I NEED TO FIND 360 day CLV of when they joined! 
transactions['average_spend'] = transactions.groupby('customer_id')['transaction_amount'].mean()
transactions

,transaction_id,customer_id,transaction_date,transaction_amount,product_category,average_spend
0,1,1,2024-02-03,165.56,Clothing,NaN
1,2,1,2024-03-02,699.01,Home Goods,584.913333
2,3,1,2024-03-12,146.86,Home Goods,675.702222
3,4,1,2024-01-20,927.46,Electronics,242.478333
4,5,1,2024-02-25,1395.87,Electronics,393.734000
...,...,...,...,...,...,...
129540,129541,10000,2023-05-27,371.70,Home Goods,NaN
129541,129542,10000,2023-06-11,1245.23,Electronics,NaN
129542,129543,10000,2023-06-28,745.74,Electronics,NaN
129543,129544,10000,2023-07-21,587.37,Home Goods,NaN


In [1353]:
latest_date

Timestamp('2024-05-31 00:00:00')

## RFM SEGMENTATION

In [1354]:
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'])
customers['join_date'] = pd.to_datetime(customers['join_date'])

In [1355]:
#Defining reference date to be the most recent date (May 31, 2024)

latest_purchase = transactions['transaction_date'].max()
latest_purchase

Timestamp('2024-05-31 00:00:00')

In [1356]:
first_purchase = transactions.groupby('customer_id')['transaction_date'].min()
first_purchase

customer_id
1       2024-01-09
2       2021-09-29
3       2021-09-28
4       2022-01-11
5       2022-03-02
           ...    
9996    2023-01-01
9997    2022-07-15
9998    2023-11-13
9999    2022-05-24
10000   2023-05-27
Name: transaction_date, Length: 10000, dtype: datetime64[ns]

In [1357]:
# Filter transactions for the first 360 days since joining -- TIMING FUNCTION ** PLAY WITH THIS MODEL TO CALCULATE ANNUALIZED, WEEKLY
# Find out... From their first transaction, their CLV 360 after -- I should be able to do this from their first purchase AND LTV since they signed up and joined
time_period_days = 360

last_360_days = latest_date - pd.Timedelta(days=time_period_days)
recent_transactions = transactions[transactions['transaction_date'] <= last_360_days]


#DECEMBER 3, 2023 reference date
recent_transactions

,transaction_id,customer_id,transaction_date,transaction_amount,product_category,average_spend
6,7,2,2022-05-05,443.99,Electronics,603.202353
7,8,2,2023-01-03,492.16,Clothing,267.964000
8,9,2,2021-12-11,910.21,Electronics,780.708000
9,10,2,2022-10-20,42.29,Clothing,471.952727
10,11,2,2021-09-29,1866.56,Electronics,372.640000
...,...,...,...,...,...,...
129531,129532,9999,2022-07-14,171.52,Electronics,NaN
129532,129533,9999,2022-06-25,134.40,Clothing,NaN
129533,129534,9999,2022-05-24,1467.78,Electronics,NaN
129534,129535,9999,2022-07-06,761.51,Electronics,NaN


In [1358]:
first_360_days = first_purchase + pd.Timedelta(days=time_period_days)
first_360_days

customer_id
1       2025-01-03
2       2022-09-24
3       2022-09-23
4       2023-01-06
5       2023-02-25
           ...    
9996    2023-12-27
9997    2023-07-10
9998    2024-11-07
9999    2023-05-19
10000   2024-05-21
Name: transaction_date, Length: 10000, dtype: datetime64[ns]

In [1359]:
historical_transactions = transactions[transactions['transaction_date'] <= last_360_days]
historical_transactions

,transaction_id,customer_id,transaction_date,transaction_amount,product_category,average_spend
6,7,2,2022-05-05,443.99,Electronics,603.202353
7,8,2,2023-01-03,492.16,Clothing,267.964000
8,9,2,2021-12-11,910.21,Electronics,780.708000
9,10,2,2022-10-20,42.29,Clothing,471.952727
10,11,2,2021-09-29,1866.56,Electronics,372.640000
...,...,...,...,...,...,...
129531,129532,9999,2022-07-14,171.52,Electronics,NaN
129532,129533,9999,2022-06-25,134.40,Clothing,NaN
129533,129534,9999,2022-05-24,1467.78,Electronics,NaN
129534,129535,9999,2022-07-06,761.51,Electronics,NaN


In [1360]:
## Shows theres 4124 transactions included for the transactionss within the last 30 days
recent_transactions

,transaction_id,customer_id,transaction_date,transaction_amount,product_category,average_spend
6,7,2,2022-05-05,443.99,Electronics,603.202353
7,8,2,2023-01-03,492.16,Clothing,267.964000
8,9,2,2021-12-11,910.21,Electronics,780.708000
9,10,2,2022-10-20,42.29,Clothing,471.952727
10,11,2,2021-09-29,1866.56,Electronics,372.640000
...,...,...,...,...,...,...
129531,129532,9999,2022-07-14,171.52,Electronics,NaN
129532,129533,9999,2022-06-25,134.40,Clothing,NaN
129533,129534,9999,2022-05-24,1467.78,Electronics,NaN
129534,129535,9999,2022-07-06,761.51,Electronics,NaN


In [1361]:
# Looking at all the transactions 6 months before the latest date
historical_transactions.describe()

,transaction_id,customer_id,transaction_date,transaction_amount,average_spend
count,85101.000000,85101.000000,85101,85101.000000,6535.000000
mean,64811.863315,5001.758252,2022-02-22 15:18:21.434765312,596.414587,598.366288
min,7.000000,2.000000,2020-01-06 00:00:00,10.020000,23.810000
25%,32368.000000,2492.000000,2021-07-04 00:00:00,227.690000,487.003421
50%,64782.000000,5006.000000,2022-04-05 00:00:00,438.540000,585.626842
75%,97114.000000,7472.000000,2022-11-20 00:00:00,846.940000,693.208250
max,129541.000000,10000.000000,2023-06-06 00:00:00,1999.990000,1997.080000
std,37401.377443,2882.457877,NaN,490.772029,199.366618


In [1362]:
historical_transactions['transaction_date']

6        2022-05-05
7        2023-01-03
8        2021-12-11
9        2022-10-20
10       2021-09-29
            ...    
129531   2022-07-14
129532   2022-06-25
129533   2022-05-24
129534   2022-07-06
129540   2023-05-27
Name: transaction_date, Length: 85101, dtype: datetime64[ns]

In [1363]:
#How many transactions per customer in referencing anything before March 30th 
historical_transactions['customer_id'].value_counts().reset_index()

,customer_id,count
0,8287,25
1,3865,25
2,9550,25
3,9260,25
4,5411,25
...,...,...
7573,2252,1
7574,2250,1
7575,8264,1
7576,2230,1


In [1364]:
# Calculate RFM metrics *** Calculate from first_transaction to latest one from December 3, 2023
rfm_df = historical_transactions.groupby('customer_id').agg({
    'transaction_date' : lambda x: (last_360_days - x.min()).days,
    'transaction_id' : 'count',
    'transaction_amount' : 'sum'
}).reset_index()

In [1365]:
rfm_df.columns = ['customer_id', 'recency', 'frequency', 'monetary']

In [1366]:
monetary_df = historical_transactions.groupby('customer_id')['transaction_amount'].sum().reset_index()
monetary_df.rename(columns={'transaction_amount': 'monetary'}, inplace=True)

monetary_df

,customer_id,monetary
0,2,6081.32
1,3,1454.87
2,4,7874.68
3,5,15524.55
4,6,10254.44
...,...,...
7573,9995,9005.05
7574,9996,4401.95
7575,9997,5848.30
7576,9999,6721.86


In [1367]:
# Assign RFM scores (using quintiles)
rfm_df['recency_score'] = pd.qcut(rfm_df['recency'], 5, labels=[5, 4, 3, 2, 1],duplicates='drop')
rfm_df['frequency_score'] = pd.qcut(rfm_df['frequency'], 5, labels=[1, 2, 3, 4, 5],duplicates='drop')
rfm_df['monetary_score'] = pd.qcut(rfm_df['monetary'], 5, labels=[1, 2, 3, 4, 5],duplicates='drop')

# Combine RFM scores into a single score
rfm_df['RFM_score'] = rfm_df[['recency_score', 'frequency_score', 'monetary_score']].sum(axis=1)


In [1368]:
rfm_df['last_purchase'] = merged_df['last_purchase_date']

In [1369]:
rfm_df

,customer_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_score,last_purchase
0,2,615,9,6081.32,3,3,3,9,NaN
1,3,616,6,1454.87,3,2,1,6,2024-03-17
2,4,511,20,7874.68,3,5,4,12,2023-10-25
3,5,461,24,15524.55,3,5,5,13,2022-11-27
4,6,488,17,10254.44,3,4,4,11,2022-09-01
...,...,...,...,...,...,...,...,...,...
7573,9995,1000,16,9005.05,1,4,4,9,2023-11-19
7574,9996,156,7,4401.95,5,2,2,9,2023-12-15
7575,9997,326,12,5848.30,4,3,3,10,2023-07-25
7576,9999,378,12,6721.86,4,3,3,10,2021-08-12


## Classification Model

In [1370]:
rfm_df['LTV'] = monetary_df['monetary']

In [1371]:
# Convert CLV into categories (e.g., low, medium, high) **Based on Lifetime CLV
rfm_df['CLV_category'] = pd.qcut(rfm_df['LTV'], 3, labels=['low', 'medium', 'high'])

In [1372]:
rfm_df['average_spend'] = transactions['average_spend']

In [1373]:
# Shows overview of RFM framework of historical data for anything earlier than December 3, 2023
rfm_df

,customer_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_score,last_purchase,LTV,CLV_category,average_spend
0,2,615,9,6081.32,3,3,3,9,NaN,6081.32,medium,NaN
1,3,616,6,1454.87,3,2,1,6,2024-03-17,1454.87,low,584.913333
2,4,511,20,7874.68,3,5,4,12,2023-10-25,7874.68,medium,675.702222
3,5,461,24,15524.55,3,5,5,13,2022-11-27,15524.55,high,242.478333
4,6,488,17,10254.44,3,4,4,11,2022-09-01,10254.44,high,393.734000
...,...,...,...,...,...,...,...,...,...,...,...,...
7573,9995,1000,16,9005.05,1,4,4,9,2023-11-19,9005.05,high,575.671250
7574,9996,156,7,4401.95,5,2,2,9,2023-12-15,4401.95,medium,607.929091
7575,9997,326,12,5848.30,4,3,3,10,2023-07-25,5848.30,medium,1048.046667
7576,9999,378,12,6721.86,4,3,3,10,2021-08-12,6721.86,medium,96.765000


In [1374]:
rfm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7578 entries, 0 to 7577
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   customer_id      7578 non-null   int64   
 1   recency          7578 non-null   int64   
 2   frequency        7578 non-null   int64   
 3   monetary         7578 non-null   float64 
 4   recency_score    7578 non-null   category
 5   frequency_score  7578 non-null   category
 6   monetary_score   7578 non-null   category
 7   RFM_score        7578 non-null   int64   
 8   last_purchase    7577 non-null   object  
 9   LTV              7578 non-null   float64 
 10  CLV_category     7578 non-null   category
 11  average_spend    7577 non-null   float64 
dtypes: category(4), float64(3), int64(4), object(1)
memory usage: 504.1+ KB


In [1375]:
X = rfm_df[['recency_score', 'frequency_score', 'monetary_score']]
y = rfm_df['CLV_category']

In [1376]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [1377]:
model = RandomForestClassifier(n_estimators=100, random_state=1234)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=1234)

In [1378]:
# Make predictions
y_pred = model.predict(X_test)

In [1379]:
y_pred

array(['medium', 'low', 'low', ..., 'medium', 'high', 'low'], dtype=object)

In [1380]:
y_test

1337    medium
207        low
2216       low
1521    medium
5412    medium
         ...  
6007       low
5407       low
5434    medium
1702      high
3541       low
Name: CLV_category, Length: 1516, dtype: category
Categories (3, object): ['low' < 'medium' < 'high']

In [1381]:
rfm_df[rfm_df['customer_id'] ==  2871]

,customer_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_score,last_purchase,LTV,CLV_category,average_spend
2172,2871,621,6,4635.27,3,2,2,7,2022-10-20,4635.27,medium,569.9115


In [1382]:
# Evaluate the model
precision = precision_score(y_test, y_pred, average='weighted')
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Precision: {precision}")
print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

print(classification_report(y_test, y_pred))

Precision: 0.8596304947069544
Accuracy: 0.8562005277044855
Recall: 0.8562005277044855
F1-Score: 0.8507365679173012
              precision    recall  f1-score   support

        high       0.83      0.96      0.89       493
         low       0.86      0.95      0.91       499
      medium       0.88      0.67      0.76       524

    accuracy                           0.86      1516
   macro avg       0.86      0.86      0.85      1516
weighted avg       0.86      0.86      0.85      1516



In [1383]:
#States scores based on importance -- LOOKS LIKE MONETARY HAS THE MOST IMPORTANCE
feature_importances = pd.DataFrame(model.feature_importances_,
                                   index = X_train.columns,
                                   columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

                 importance
monetary_score     0.578130
frequency_score    0.391012
recency_score      0.030858


In [1384]:
rfm_df = rfm_df.drop(columns='CLV_category')

## Regression Model

In [1385]:
X = rfm_df[['recency_score', 'frequency_score', 'monetary_score']]
y = rfm_df['average_spend']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [1386]:
model_2 = RandomForestRegressor(n_estimators=100, random_state=42)
model_2.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [1387]:
#New Predictor for the regression model
y_pred = model_2.predict(X_test)


In [1388]:
y_pred

array([592.24275308, 611.60773842, 593.16276025, ..., 614.45387958,
       603.95925966, 614.45387958])

In [1389]:
# Evaluate the model - Right now this model looks close to perfect which is incorrect
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")

ValueError: Input contains NaN.

## CUSTOM REGRESSION